In [51]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [52]:
dataset = load_digits()
# dataset

In [53]:
dataset.data.shape

(1797, 64)

In [54]:
dataset.images.shape

(1797, 8, 8)

In [55]:
dataset.target.shape

(1797,)

In [56]:
X = dataset.data
Y = dataset.target
Y = np.eye(10)[Y]  #one hot #  [Y]: به عنوان اندیس آی (10) استفاده میشود

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((1437, 64), (360, 64), (1437, 10), (360, 10))

In [57]:
def sigmoid(X):
    return 1 / (1 + np.exp(-X))

def softmax(X):  #در شبکه عصبی که مسیله کلاسیفیکیشین باشد لایه ی آخر از سافتمکس استفاده میشود 
    return np.exp(X) / np.sum(np.exp(X))

def root_mean_squired_error(Y_gt, Y_pred):
    return np.sqrt(np.mean((Y_gt - Y_pred) ** 2))


In [58]:
epochs = 80
η = 0.001  # learning rate

D_in = X_train.shape[1]  # 64 #لایه ورودی
H1 = 128  # لایه پنهان اول
H2 = 32   # لایه پنهان دوم
D_out = Y_train.shape[1]  # 10  # لایه خروجی

In [59]:
W1 = np.random.randn(D_in, H1)  #ماتریس اتصالات
W2 = np.random.randn(H1, H2) 
W3 = np.random.randn(H2, D_out) 

In [60]:
B1 = np.random.randn(1, H1)  
B2 = np.random.randn(1, H2)
B3 = np.random.randn(1, D_out)

In [61]:
for epoch in range(epochs):
#train
    Y_pred = []
    for x, y in zip(X_train, Y_train):
        x = x.reshape(-1, 1)
        # forward

        #layer 1
        out1 = sigmoid(x.T @ W1 + B1)

        #layer 2
        out2 = sigmoid(out1 @ W2 + B2)

        #layer 3
        out3 = softmax(out2 @ W3 + B3)
        y_pred = out3
        Y_pred.append(y_pred)

        loss = root_mean_squired_error(y, y_pred)

        # backward     #باید همش مشتق بگیریم

        #layer 3
        error = -2 * (y - y_pred) # مشتق root_mean_squired_error
        grad_B3 = error
        grad_W3 = out2.T @ error

        #layer 2
        error = error @ W3.T * out2 * (1 - out2) # مشتق سیگموید : out2 * (1 - out2)
        grad_B2 = error
        grad_W2 = out1.T @ error

        #layer 1
        error = error @ W2.T * out1*(1 - out1) 
        grad_B1 = error
        grad_W1 = x @ error

        # update

        #layer 1
        W1 -= η * grad_W1
        B1 -= η * grad_B1

        #layer 2
        W2 -= η * grad_W2
        B2 -= η * grad_B2

        #layer 3
        W3 -= η * grad_W3
        B3 -= η * grad_B3

    Y_pred = np.array(Y_pred).reshape(-1, 10)   
    loss_train = root_mean_squired_error(Y_train, Y_pred)
    accuracy_train = np.sum(np.argmax(Y_train, axis=1) == np.argmax(Y_pred, axis=1)) / len(Y_train)
    
    print("loss_train:", loss_train)
    print("accuracy_train:", accuracy_train)

      # test

    Y_pred = []
    for x, y in zip(X_test, Y_test):

        # forward
        x = x.reshape(-1, 1)

        # layer 1
        net1 = x.T @ W1 + B1
        out1 = sigmoid(net1)

        # layer 2
        net2 = out1 @ W2 + B2
        out2 = sigmoid(net2)

        # layer 3
        net3 = out2 @ W3 + B3
        out3 = softmax(net3)

        y_pred = out3
        Y_pred.append(y_pred.T)

    Y_pred = np.array(Y_pred).reshape(-1, 10)
    loss_test = root_mean_squired_error(Y_pred, Y_test)
    accuracy_test = np.mean(np.argmax(Y_pred, axis=1) == np.argmax(Y_test, axis=1))

    print("loss_test:", loss_test)
    print("accuracy_test:", accuracy_test)

loss_train: 0.3366298641512468
accuracy_train: 0.10508002783576896
loss_test: 0.31396808319408187
accuracy_test: 0.15833333333333333
loss_train: 0.3000158301491934
accuracy_train: 0.23312456506610996
loss_test: 0.2913894444190616
accuracy_test: 0.2972222222222222
loss_train: 0.278458003709141
accuracy_train: 0.360473208072373
loss_test: 0.27497377352019364
accuracy_test: 0.4083333333333333
loss_train: 0.2604956629554922
accuracy_train: 0.49269311064718163
loss_test: 0.2608534292737929
accuracy_test: 0.5305555555555556
loss_train: 0.24707423446940593
accuracy_train: 0.5553235908141962
loss_test: 0.2502976372873518
accuracy_test: 0.5666666666666667
loss_train: 0.23499865703313572
accuracy_train: 0.6137787056367432
loss_test: 0.2418258009646612
accuracy_test: 0.5888888888888889
loss_train: 0.22503572414921447
accuracy_train: 0.6687543493389004
loss_test: 0.23448709357963823
accuracy_test: 0.6111111111111112
loss_train: 0.2160673898336548
accuracy_train: 0.7028531663187195
loss_test: 0.228

In [62]:
# predict

import cv2

image = cv2.imread("4.png")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image = image.astype(np.float32)

x = image.reshape(-1, 1)

# layer 1
net1 = x.T @ W1 + B1
out1 = sigmoid(net1)

# layer 2
net2 = out1 @ W2 + B2
out2 = sigmoid(net2)

# layer 3
net3 = out2 @ W3 + B3
out3 = softmax(net3)

y_pred = out3
print(np.argmax(y_pred))

9
